Створіть клас CustomDataset, який буде відповідати за завантаження та підготовку даних. Ви можете створити простий шаблон датасету, який завантажує дані з файлів або створює штучні дані. Звертайтеся до відповідного розділу документації: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
#шаблон для завантаження з файлу
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [4]:
#шаблон для створення штучних даних
class SyntheticDataset(Dataset):
    def __init__(self, num_samples, data_shape, transform=None):
        self.num_samples = num_samples
        self.data_shape = data_shape
        self.transform = transform

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        data = torch.randn(self.data_shape)
        label = torch.sum(data)
        sample = {'data': data, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

Напишіть клас DataLoader, який буде відповідати за подачу даних в модель. Використовуйте пакети даних, щоб ефективно завантажувати дані та передавати їх у модель батчами. Звертайтеся до відповідного розділу документації: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [6]:
class CustomDataLoader:
    
    def __init__(self, dataset, batch_size=32, shuffle=True, num_workers=0):
        """
        Args:
            dataset (Dataset): Набір даних, який буде подаватися у модель.
            batch_size (int): Розмір батчу.
            shuffle (bool): Якщо True, дані будуть перемішані.
            num_workers (int): Кількість процесів для завантаження даних.
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers

        # Використовуємо вбудований DataLoader
        self.dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    def __iter__(self):
        # Ітерація через DataLoader
        for batch in self.dataloader:
            yield batch

    def __len__(self):
        # Повертає кількість батчів у наборі даних
        return len(self.dataloader)

In [7]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = CustomDataLoader(training_data, batch_size=64)
test_dataloader = CustomDataLoader(test_data, batch_size=64)

Створіть базову модель за допомогою модуля torch.nn. Модель може бути дуже простою, наприклад, одношаровою нейронною мережею. Звертайтеся до відповідного розділу документації: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [9]:
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
# print(f"Using {device} device")

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits

In [11]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


Напишіть цикли навчання (train_loop) та валідації моделі (valid_loop). Ці цикли мають включати процес передачі даних у модель, обчислення втрат та оновлення параметрів моделі. Звертайтеся до відповідного розділу документації: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

Виберіть оптимізатор та налаштуйте його для оптимізації параметрів моделі. Звертайтеся до відповідного розділу документації: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303175  [   64/60000]
loss: 2.242201  [ 6464/60000]
loss: 2.192909  [12864/60000]
loss: 2.126072  [19264/60000]
loss: 2.054506  [25664/60000]
loss: 1.965616  [32064/60000]
loss: 1.934055  [38464/60000]
loss: 1.880925  [44864/60000]
loss: 1.856179  [51264/60000]
loss: 1.812169  [57664/60000]
Test Error: 
 Accuracy: 63.8%, Avg loss: 1.780009 

Epoch 2
-------------------------------
loss: 1.789651  [   64/60000]
loss: 1.682480  [ 6464/60000]
loss: 1.605309  [12864/60000]
loss: 1.677829  [19264/60000]
loss: 1.488337  [25664/60000]
loss: 1.448775  [32064/60000]
loss: 1.501209  [38464/60000]
loss: 1.459483  [44864/60000]
loss: 1.489749  [51264/60000]
loss: 1.339190  [57664/60000]
Test Error: 
 Accuracy: 65.4%, Avg loss: 1.366549 

Epoch 3
-------------------------------
loss: 1.368229  [   64/60000]
loss: 1.320738  [ 6464/60000]
loss: 1.329946  [12864/60000]
loss: 1.339430  [19264/60000]
loss: 1.135606  [25664/60000]
loss: 1.291558  [32064/600